<a href="https://colab.research.google.com/github/Rt247/Not_NLP_CW/blob/BERT_method/NLP_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib
import json

url_hash = "384830ba34a2c9a1dfeb869062d8c6dc/raw/729c688d2bb4801a135a1f3824f30311a84de788"

data_set_load = {
  "url_dev_enzh_mt": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/dev.enzh.mt',
  "url_dev_enzh_scores": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/dev.enzh.scores',
  "url_dev_enzh_src": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/dev.enzh.src',
  "url_train_enzh_mt": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/train.enzh.mt',
  "url_train_enzh_scores": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/train.enzh.scores',
  "url_train_enzh_src": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/train.enzh.src',
  "url_test_enzh_src": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/test.enzh.src',
  "url_test_enzh_mt": f'https://gist.githubusercontent.com/cheukwing/{url_hash}/test.enzh.mt',
}

data_sets = {}
for key, value in data_set_load.items():
  r = urllib.request.urlopen(value)
  data = r.read().splitlines()
  data_sets[key[4:]] = data
  print(data_sets[key[4:]][0])

print(data_sets.keys())

b'\xe6\xb1\x89\xe5\xaf\x86\xe5\xb0\x94\xe9\xa1\xbf\xe5\x9c\xa8\xe4\xb9\x94\xe6\xb2\xbb \xc2\xb7 \xe5\x8d\x8e\xe7\x9b\x9b\xe9\xa1\xbf\xe7\x9a\x84\xe9\x80\x89\xe4\xbb\xbb\xe7\xbb\x9f\xe6\xb2\xbb\xe4\xb8\x8b\xe7\x9b\x91\xe7\x9d\xa3\xe4\xbb\x96\xe7\x9a\x84\xe5\x90\x8c\xe4\xba\x8b\xe4\xbb\xac.'
b'-1.0228233810174194'
b'Hamilton oversaw his colleagues under the elective reign of George Washington.'
b'\xe6\x9c\x80\xe5\x90\x8e\xe7\x9a\x84\xe5\xbe\x81\xe6\x9c\x8d\xe8\x80\x85\xe9\xaa\x91\xe7\x9d\x80\xe4\xbb\x96\xe7\x9a\x84\xe5\x89\x91\xe7\xbb\xa7\xe7\xbb\xad\xe5\x89\x8d\xe8\xbf\x9b.'
b'-1.5284005772625449'
b'The last conquistador then rides on with his sword drawn.'
b'Milhaud also used jazz idioms, as in his Suite for clarinet, violin and piano.'
b'\xe7\xb1\xb3\xe5\x8b\x92\xe8\xb1\xaa\xe5\xbe\xb7\xe4\xb9\x9f\xe4\xbd\xbf\xe7\x94\xa8\xe7\x88\xb5\xe5\xa3\xab\xe4\xb9\x90\xe6\x88\x90\xe8\xaf\xad, \xe5\xb0\xb1\xe5\x83\x8f\xe4\xbb\x96\xe7\x9a\x84\xe5\xa5\x97\xe6\x88\xbf\xe9\x87\x8c\xe4\xbd\xbf\xe7\x94\

In [2]:
!pip install transformers
import torch
from transformers import BertTokenizer, BertModel

     |████████████████████████████████| 481kB 2.8MB/s 
     |████████████████████████████████| 870kB 39.7MB/s 
     |████████████████████████████████| 3.1MB 35.0MB/s 
     |████████████████████████████████| 1.0MB 21.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=35c18099d03d9272ffb4eeaded8163eed13e699c0ba9b9cf027e5bda90cb19c8
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [22]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenize our sentence with the BERT tokenizer.
original_texts = [text.decode() for text in data_sets['train_enzh_src']]
translated_texts = [text.decode('utf8') for text in data_sets['train_enzh_mt']]

text_pairs = list(zip(original_texts, translated_texts))


inputs = [tokenizer.encode_plus(original, text_pair=translated, add_special_tokens = True, pad_to_max_length=True) for original, translated in text_pairs]

input_ids = [d['input_ids'] for d in inputs]
input_segments = [d['token_type_ids'] for d in inputs]
input_attention_masks = [d['attention_mask'] for d in inputs]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor(input_ids)
segments_tensor = torch.tensor(input_segments)
attention_masks_tensor = torch.tensor(input_attention_masks)

# # Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-multilingual-cased')

# # Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [0]:
# # Predict hidden states features for each layer
# with torch.no_grad():
#     last_hidden_states = model(tokens_tensor, segments_tensors)

# features = last_hidden_states[0][:][0][:].numpy()
# print(features)